<a href="https://colab.research.google.com/github/hugomaharavo/ai4eng-20242-Pruebas-Saber-Pro-Colombia/blob/main/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ACCEDER A LOS ARCHIVOS EN KAGGLE
# 1.1 - Instale la biblioteca opendatasets para descargar conjuntos de datos directamente de Kaggle
!pip install opendatasets

# 1.2 - Importar los módulos necesarios
import opendatasets as kaggle_data
import os

# 1.3 - Enlace al concurso de Kaggle
competition_url = "https://www.kaggle.com/competitions/udea-ai4eng-20242"

# 1.4 - Descargar los datos del concurso
kaggle_data.download(competition_url)

# 1.5 - Cambiar de directorio para acceder a la carpeta del conjunto de datos descargado
project_dir = "udea-ai4eng-20242"
os.chdir(project_dir)

# 1.6 - Comprobación de los archivos descargados
files = os.listdir()
print("Fichiers disponibles dans le dossier : ", files)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: hugomaharavo
Your Kaggle Key: ··········


100%|██████████| 20.1M/20.1M [00:00<00:00, 100MB/s] 



Extracting archive ./udea-ai4eng-20242/udea-ai4eng-20242.zip to ./udea-ai4eng-20242
Fichiers disponibles dans le dossier :  ['train.csv', 'test.csv', 'submission_example.csv']


In [5]:
# 2 - IMPORTAR BIBLIOTECAS
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

# 3 - CARGAR LOS DATOS INICIALES
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# 4 - EXCLUIR COLUMNAS DE ALTA CARDINALIDAD
train_df = train_df.drop(columns=['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO'])
test_df = test_df.drop(columns=['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO'])

# 5 - APLICAR MAPAS Y CONVERSIONES
# 5.1 - ESTU_VALORMATRICULAUNIVERSIDAD
tuition_average_mapping = {
    "No pagó matrícula": 0.0, "Menos de 500 mil": 0.25, "Entre 500 mil y menos de 1 millón": 0.75,
    "Entre 1 millón y menos de 2.5 millones": 1.75, "Entre 2.5 millones y menos de 4 millones": 3.25,
    "Entre 4 millions et moins de 5.5 millions": 4.75, "Entre 5.5 millions et moins de 7 millions": 6.25, "Más de 7 millones": 7.5
}
train_df['ESTU_VALORMATRICULAUNIVERSIDAD'] = train_df['ESTU_VALORMATRICULAUNIVERSIDAD'].map(tuition_average_mapping)
test_df['ESTU_VALORMATRICULAUNIVERSIDAD'] = test_df['ESTU_VALORMATRICULAUNIVERSIDAD'].map(tuition_average_mapping)

# 5.2 - FAMI_ESTRATOVIVIENDA
housing_strata_mapping = {"Estrato 1": 1, "Estrato 2": 2, "Estrato 3": 3, "Estrato 4": 4, "Estrato 5": 5, "Estrato 6": 6, "Sin Estrato": None}
train_df['FAMI_ESTRATOVIVIENDA'] = train_df['FAMI_ESTRATOVIVIENDA'].map(housing_strata_mapping)
test_df['FAMI_ESTRATOVIVIENDA'] = test_df['FAMI_ESTRATOVIVIENDA'].map(housing_strata_mapping)

# 5.3 - FAMI_TIENEINTERNET et ESTU_PAGOMATRICULAPROPIO
binary_mappings = {"Si": 1, "No": 0}
train_df['FAMI_TIENEINTERNET'] = train_df['FAMI_TIENEINTERNET'].map(binary_mappings)
test_df['FAMI_TIENEINTERNET'] = test_df['FAMI_TIENEINTERNET'].map(binary_mappings)
train_df['ESTU_PAGOMATRICULAPROPIO'] = train_df['ESTU_PAGOMATRICULAPROPIO'].map(binary_mappings)
test_df['ESTU_PAGOMATRICULAPROPIO'] = test_df['ESTU_PAGOMATRICULAPROPIO'].map(binary_mappings)

# 5.4 - Codificación de la variable objetivo
target_mapping = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
train_df['RENDIMIENTO_GLOBAL'] = train_df['RENDIMIENTO_GLOBAL'].map(target_mapping)

# 5.5 - Separación de características y variable objetivo
X_train = train_df.drop(columns=['ID', 'RENDIMIENTO_GLOBAL'])
y_train = train_df['RENDIMIENTO_GLOBAL']
X_test = test_df.drop(columns=['ID'])


# 6 - PROCESO PARA PRETRATAMIENTO
# 6.1 - Identificación de columnas de baja cardinalidad para OneHotEncoder
low_cardinality_cols = [col for col in X_train.columns if X_train[col].nunique() < 10 and X_train[col].dtype == "object"]

# 6.2 - Proceso de pretratamiento simplificada
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), X_train.select_dtypes(include=['float64', 'int64']).columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), low_cardinality_cols)
    ]
)

# 6.3 - Proceso de entrenamiento con un modelo de árbol de decisión sin hiperparámetros
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier())
])

# 7 - ENTRENAR EL MODELO
pipeline.fit(X_train, y_train)

# 8 - HACER PREDICCIONES SOBRE EL CONJUNTO DE DATOS test
predictions = pipeline.predict(X_test)

# 9 - CONVERTIR LAS PREDICCIONES EN LA REDACCIÓN ORIGINAL
target_inverse_mapping = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}
predictions_labels = [target_inverse_mapping[pred] for pred in predictions]

# 10 - CREA EL ARCHIVO DE PRESENTACIÓN, y asegúrate de que las dimensiones son consistentes con las requeridas por la competición de Kaggle.
submission_df = pd.DataFrame({
    "ID": test_df['ID'],
    "RENDIMIENTO_GLOBAL": predictions_labels
})
submission_df.to_csv("submission.csv", index=False)

print("Dimensiones de la clasificación submission.csv (filas, columnas) ", submission_df.shape)

Dimensiones de la clasificación submission.csv (filas, columnas)  (296786, 2)


In [6]:
# 11 - DESCARGAR EL ARCHIVO
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>